In [ ]:
# Colab setup code block
try:
    import google.colab
    print("Running on Google Colab. Setting up environment...")
    
    import os
    if not os.path.exists('CMAI-E91-Students'):
        !git clone https://github.com/algolab-quantique/CMAI-E91-Students.git

    os.chdir('CMAI-E91-Students/workshops')
    !pip install -r ../requirements.txt -q
    print("Environment setup complete!")
except ImportError:
    print("Not running on Google Colab. Skipping setup.")


# CMAI E91 Atelier sur la Distribution Quantique de Clés

**Durée :** 3 heures (théorie + pratique + devoir)  
**Niveau :** Débutant  

Ce notebook d'atelier unifié couvre :
1. **Partie 1 :** Comprendre l'intrication et l'inégalité de Bell CHSH
2. **Partie 2 :** Mise en œuvre du protocole complet E91 QKD
3. **Partie 3 :** Défi de codage (Devoir)

---

# Inégalité de Bell - CHSH - Atelier Pratique

**Objectifs pédagogiques :**
- Créer des états de Bell intriqués  
- Implémenter des transformations de base de mesure  
- Calculer les corrélations et la valeur de CHSH  
- Vérifier que l’intrication quantique viole les limites classiques  

**Objectif :** Montrer que des paires intriquées peuvent produire $|S| > 2$.

**Feuille de route (fonctions à implémenter) :**
`create_bell_pair_singlet_state()` → `apply_basis_transformation()` → `measure_bell_pair()` → `organize_measurements_by_basis()` → `calculate_correlations()` → `calculate_chsh_value()`

---

## Inégalité de Bell (CHSH) : points clés (référence rapide)

Le test CHSH vérifie si deux systèmes (par exemple des qubits) sont intriqués.

### Ce que signifie la valeur CHSH

| Système | Valeur CHSH $\|S\|$ |
|--------|-------------|
| Classique (variables cachées) | $\|S\| \le 2$ |
| **Quantique (intriqué)** | **$2 < \|S\| \le 2\sqrt{2} \approx 2.83$** |

Si $|S| > 2$ → **Intrication quantique confirmée !**

En pratique, une violation significative (au-delà des barres d’erreur) est nécessaire pour conclure expérimentalement à l’intrication.


### Quelles bases de mesure sont utilisées ?

Le test utilise **quatre combinaisons** de bases de mesure pour Alice et Bob. Chaque côté utilise **deux bases** :
- **Alice :** $a_1 = 0^\circ$, $a_2 = 90^\circ$
- **Bob :** $b_1 = 45^\circ$, $b_2 = 135^\circ$

Paires utilisées dans le CHSH : $(a_1,b_1)$, $(a_1,b_2)$, $(a_2,b_1)$, $(a_2,b_2)$

Les bases sont appliquées par `apply_basis_transformation()` et les mesures sont effectuées dans `measure_bell_pair()`.

### Corrélation pour chaque paire de bases

Calculée par `calculate_correlations()`

$$
E(a,b) = \frac{N_{00} + N_{11} - N_{01} - N_{10}}{N_{00} + N_{11} + N_{01} + N_{10}}
$$

où $N_{xy}$ est le **nombre d’occurrences** du résultat de mesure $x$ pour Alice et $y$ pour Bob (par exemple, $N_{00}$ est le nombre de fois où le résultat était $00$).

Regroupement effectué par `organize_measurements_by_basis()`.

### Formule CHSH

$$
S = E(a_1,b_1) - E(a_1,b_2) + E(a_2,b_1) + E(a_2,b_2)
$$

Calculée par `calculate_chsh_value()`.

---


## Pont entre la présentation et le notebook : de la physique à la simulation

**Expérience réelle :** photons intriqués + polariseurs / lames à retard qui fixent les angles de mesure.  
**En pratique :** on fait pivoter des éléments optiques pour choisir la base de mesure.  
**Dans ce notebook :** paires de Bell + portes de rotation pour simuler ces angles avec Qiskit.

Ainsi, lorsque vous voyez **0°, 45°, 90°, 135°** ci-dessous, pensez aux **angles des polariseurs** au laboratoire.

---


## Configuration

Exécutez cette cellule pour importer toutes les bibliothèques et fonctions auxiliaires requises.

In [1]:
# Standard imports
import random
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Set seed for reproducibility
GLOBAL_SEED = 91
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

# Initialize simulator
aer_simulator = AerSimulator()
MANUAL_SIMULATOR_SEED_COUNTER = GLOBAL_SEED

# Constants
ALICE_BELL_BASES = ['0', '90']   # Alice's measurement bases for CHSH
BOB_BELL_BASES = ['45', '135']   # Bob's measurement bases for CHSH
BELL_INEQUALITY_THRESHOLD = 2.0  # Classical limit

print("Setup complete!")
print(f"Alice's bases: {ALICE_BELL_BASES}")
print(f"Bob's bases: {BOB_BELL_BASES}")

Setup complete!
Alice's bases: ['0', '90']
Bob's bases: ['45', '135']


In [2]:
# Helper function to run circuits (provided - just run this cell)
def run_circuit(circ: QuantumCircuit, shots=1) -> dict:
    """
    Run a quantum circuit on the AerSimulator and return the counts.
    """
    global MANUAL_SIMULATOR_SEED_COUNTER
    global aer_simulator
    
    current_run_seed = MANUAL_SIMULATOR_SEED_COUNTER
    MANUAL_SIMULATOR_SEED_COUNTER += 1
    
    circ = transpile(circ, aer_simulator)
    result = aer_simulator.run(circ, shots=shots, seed_simulator=current_run_seed).result()
    return result.get_counts(circ)

print("Helper function loaded!")

Helper function loaded!


---
## 1) Créer un état de Bell

Nous commençons par préparer une **paire intriquée**. L'état singulet (singlet state) garantit une anti‑corrélation parfaite, ce qui entraîne une violation CHSH.

L'**état singulet** $|\Psi^-\rangle = (|01\rangle - |10\rangle)/\sqrt{2}$ est utilisé dans le protocole E91 original.

**Instructions :**
1. Appliquer la porte Hadamard (H) sur le qubit 0
2. Appliquer la porte CNOT avec contrôle = 0 et cible = 1
3. Appliquer les portes X puis Z sur le qubit 1 (pour obtenir spécifiquement l'état $|\Psi^-\rangle$)

Exercice : implémentez `create_bell_pair_singlet_state()`


In [ ]:
def create_bell_pair_singlet_state() -> QuantumCircuit:
    """
    Create the Bell singlet state |Ψ-⟩ = (|01⟩ - |10⟩)/√2
    
    Returns:
        QuantumCircuit with 2 qubits in the singlet Bell state
    """
    # TODO: Create a quantum circuit with 2 qubits
    qc = None  # Replace with: QuantumCircuit(2)
    
    # Step 1: Create superposition
    # TODO: Apply Hadamard to qubit 0
    
    # Step 2: Entangle qubits
    # TODO: Apply CNOT (control=0, target=1)
    
    # Step 3: Transform to |Ψ-⟩ state
    # TODO: Apply X gate to qubit 1
    # TODO: Apply Z gate to qubit 1
    
    return qc

In [ ]:
# Test your Bell state
bell_state = create_bell_pair_singlet_state()
bell_state.draw('mpl')

In [ ]:
# Verify by measuring many times
test_qc = bell_state.copy()
test_qc.measure_all()
counts = run_circuit(test_qc, shots=1000)
print(f"\nMeasurement results: {counts}")
print("\nExpected: roughly 50% |01⟩ and 50% |10⟩ (never |00⟩ or |11⟩)")


---
## 2) Transformation de la base de mesure

Nous mesurons toujours dans la base Z. Pour simuler d'autres bases, nous **effectuons d'abord une rotation**, puis nous mesurons.

| Base | Angle | Porte |
|------|-------|------|
| Z | 0° | — |
| 45° | 45° | Ry(-π/4) |
| X | 90° | H |
| 135° | 135° | Ry(-3π/4) |

Exercice : implémentez `apply_basis_transformation()`

### Intuition rapide (bases)

- **Même base** → forte (anti-)corrélation pour les paires intriquées.
- **Bases différentes** → les résultats semblent plus aléatoires.

C’est exactement le rôle des angles des polariseurs dans l’expérience sur les photons.

---

In [ ]:
def apply_basis_transformation(circuit: QuantumCircuit, qubit_index: int, basis: str) -> QuantumCircuit:
    """
    Apply a measurement basis transformation to a qubit.
    
    Args:
        circuit: The quantum circuit to transform
        qubit_index: Index of the qubit (0 or 1)
        basis: '0' (Z), '45', '90' (X), or '135'
    
    Returns:
        Copy of the circuit with basis transformation applied
    """
    # Create a copy of the circuit
    transformed = circuit.copy()
    
    if basis == '0':    # Z-basis (0°)
        pass  # No transformation needed
    
    elif basis == '90':  # X-basis (90°)
        # TODO: Apply Hadamard gate on the specified qubit (qubit_index)
        pass
    
    elif basis == '45':  # 45° basis
        # TODO: Apply Ry(-π/4) rotation
        # Hint: transformed.ry(-np.pi/4, qubit_index)
        pass
    
    elif basis == '135':  # 135° basis
        # TODO: Apply Ry(-3π/4) rotation
        pass
    
    else:
        raise ValueError(f"Unknown basis: {basis}")
    
    return transformed

In [ ]:
#  Test basis transformations
test_qc = QuantumCircuit(1)

print("Testing basis transformations:")
for basis in ['0', '45', '90', '135']:
    result = apply_basis_transformation(test_qc, 0, basis)
    print(f"\nBasis {basis}°:")
    print(result.draw())

---
## 3) Mesurer les paires de Bell

Chaque paire intriquée ne peut être mesurée **qu'une seule fois**, car la mesure provoque l'effondrement de l'état.

Nous combinons maintenant la création des états de Bell avec les transformations de base pour mesurer les paires intriquées.

Exercice : implémentez `measure_bell_pair()`


In [ ]:
def measure_bell_pair(circuit: QuantumCircuit, alice_basis: str, bob_basis: str) -> str:
    """
    Measure a Bell pair with Alice and Bob using specified bases.
    
    Args:
        circuit: Bell pair circuit
        alice_basis: Alice's measurement basis ('0', '45', '90')
        bob_basis: Bob's measurement basis ('45', '90', '135')
    
    Returns:
        str: Measurement result ('00', '01', '10', or '11')
    """
    # Create a copy of the circuit
    meas_qc = circuit.copy()
    
    # TODO: Apply Alice's basis transformation to qubit 0
    
    # TODO: Apply Bob's basis transformation to qubit 1
    
    # Add measurements to all qubits
    meas_qc.measure_all()
    
    # Run the circuit and return the result
    counts = run_circuit(meas_qc, shots=1)
    
    return list(counts.keys())[0]

In [ ]:
# Test Bell pair measurement
bell_state = create_bell_pair_singlet_state()
from collections import Counter


# Test multiple measurements
print("Testing Bell pair measurements (1000 runs each)")

print("\nDifferent bases (Alice 0°, Bob 45°) - Random results:")
results_diff = [measure_bell_pair(bell_state, '0', '45') for _ in range(200)]
print(f"  Counts: {dict(Counter(results_diff))}")

print("\nSame bases (Alice 0°, Bob 0°) - Perfect Anti-correlation:")
# For Singlet State |Ψ-⟩, same-basis measurements must be opposite ('01' or '10')
results_same = [measure_bell_pair(bell_state, '0', '0') for _ in range(200)]
print(f"  Counts: {dict(Counter(results_same))}")


---
## 4) Effectuer les mesures du test de Bell

Pour estimer la valeur CHSH, nous avons besoin de nombreux essais avec des **bases choisies au hasard** afin d’échantillonner les quatre paires de bases.

Pour le test CHSH, il faut :
1. Créer de nombreuses paires de Bell
2. Choisir au hasard les bases pour Alice et Bob
3. Enregistrer toutes les mesures

Exercice : implémentez `run_bell_test_measurements()`


In [ ]:
def run_bell_test_measurements(list_bell_pairs, list_alice_bases=ALICE_BELL_BASES, list_bob_bases=BOB_BELL_BASES):
    """
    Run measurements on Bell pairs using random basis choices.
    
    Args:
        list_bell_pairs: List of Bell pair circuits
        list_alice_bases: Alice's possible bases
        list_bob_bases: Bob's possible bases
    
    Returns:
        Tuple of (results, alice_bases_used, bob_bases_used)
    """
    results = []
    alice_bases_used = []
    bob_bases_used = []
    
    for qc in list_bell_pairs:
        # TODO: Randomly choose Alice's basis from list_alice_bases
        a_base = None  # Hint: random.choice(list_alice_bases)
        
        # TODO: Randomly choose Bob's basis from list_bob_bases
        b_base = None # Hint: random.choice(list_bob_bases)
        
        # TODO: Measure the Bell pair qc with a_base and b_base
        result = None  # Hint: measure_bell_pair(qc, a_base, b_base)
        
        # Record everything
        results.append(result)
        alice_bases_used.append(a_base)
        bob_bases_used.append(b_base)
    
    return results, alice_bases_used, bob_bases_used

---
## 5) Organiser les résultats par paire de bases

Nous devons **regrouper les résultats par paire de bases** afin de pouvoir calculer $E(a,b)$ pour chaque combinaison.


In [ ]:
def organize_measurements_by_basis(results, alice_bases, bob_bases):
    """
    Organize measurements by basis pairs.
    
    Returns:
        Dict mapping (alice_basis, bob_basis) to {'00': count, '01': count, ...}
    """
    # Get unique bases
    unique_alice = list(set(alice_bases))
    unique_bob = list(set(bob_bases))
    
    # Initialize counts dictionary
    counts = {}
    for a in unique_alice:
        for b in unique_bob:
            counts[(a, b)] = {'00': 0, '01': 0, '10': 0, '11': 0}
    
    # Loop through results and increment the counter for the used basis pair
    for i, result in enumerate(results):
        a_base = alice_bases[i]
        b_base = bob_bases[i]
        
        # Check if result is valid (just in case), then increment
        if result in counts[(a_base, b_base)]:
            counts[(a_base, b_base)][result] += 1
            
    return counts


---
## 6) Calculer les corrélations

La corrélation indique si Alice et Bob sont **généralement identiques ou différents**. Elle varie toujours entre **−1 et +1**.

**Formule basée sur les décomptes (utilisée dans le code) :**
$$
E(a,b)=\frac{N_{00}+N_{11}-N_{01}-N_{10}}{N_{00}+N_{11}+N_{01}+N_{10}}
$$

### Lien avec la définition des diapositives (résultats ±1)

Dans les diapositives, chaque résultat est converti en **±1**, puis moyenné :
- Bits identiques ($00$ ou $11$) → $+1$  
- Bits différents ($01$ ou $10$) → $-1$

La corrélation correspond donc simplement à la **moyenne du produit** :
$$
E(a,b)=\langle A\cdot B\rangle=\frac{1}{N}\sum_{k=1}^{N} A_k B_k
$$

La formule basée sur les décomptes ci-dessus est **exactement la même chose**, juste écrite à partir des totaux de chaque résultat.


In [ ]:
def calculate_correlations(measurements):
    """
    Calculate E(a,b) for each basis pair.
    
    Args:
        measurements: Dict from organize_measurements_by_basis()
    
    Returns:
        Dict mapping (alice_basis, bob_basis) to correlation value
    """
    correlations = {}
    
    for basis_pair, results in measurements.items():
        total = sum(results.values()) # Total measurements: results['00'] + results['01'] + results['10'] + results['11']
        
        if total > 0: # to avoid division by zero :p
              
            # Formula: (Same - Different) / Total
            E = 0 # replace with formula ...
            correlations[basis_pair] = E
        else:
            correlations[basis_pair] = 0
    
    # Print for debugging
    print("Correlations:")
    for basis, corr in correlations.items():
        print(f"  E{basis} = {corr:.4f}")
    
    return correlations


---
## 7) Calculer la valeur CHSH

Le CHSH combine les quatre corrélations selon un **schéma spécifique** qui met en évidence le comportement quantique.

**Formule :** $S = E(a_1,b_1) - E(a_1,b_2) + E(a_2,b_1) + E(a_2,b_2)$

Avec nos bases :
- $a_1 = 0^\circ$, $a_2 = 90^\circ$ (Alice)
- $b_1 = 45^\circ$, $b_2 = 135^\circ$ (Bob)

Exercice : implémentez `calculate_chsh_value()`


In [3]:
def calculate_chsh_value(correlations, alice_bases=ALICE_BELL_BASES, bob_bases=BOB_BELL_BASES):
    """
    Calculate the CHSH Bell parameter S.

    Args:
        correlations: Dictionary mapping (alice_basis, bob_basis) to correlation value E(a, b).
        example:
        correlations[(a1, b1)] = E(a1, b1)
        alice_bases: List of Alice's two measurement bases (default: ['0', '90']).
        bob_bases: List of Bob's two measurement bases (default: ['45', '135']).

    
    Returns:
        float: |S| value
    """
    a1, a2 = alice_bases  # '0', '90'
    b1, b2 = bob_bases    # '45', '135'

    # Note: correlations[(a1, b1)] gives E(a1, b1)
    
    # TODO: Calculate S using the CHSH formula
    # S = E(a1,b1) - E(a1,b2) + E(a2,b1) + E(a2,b2)
    # INDICE : Remarquez où se trouve le signe moins ! Cette formule est spécifique à l'état |Ψ-⟩.
    S = 0  # Replace with actual calculation
    
    return abs(S)


def check_bell_inequality(chsh_value):
    """Returns True if Bell inequality is violated (quantum entanglement detected)."""


## Assistant de visualisation (fourni)

Il ne fait pas partie de la logique du protocole. Utilisez-le pour visualiser clairement les corrélations et la valeur CHSH.


In [ ]:
def visualize_bell_test_results(correlations, chsh_value, title="Bell Test Results"):
    fig = plt.figure(figsize=(12, 5))
    
    # Correlation heatmap
    ax1 = fig.add_subplot(121)
    alice_bases = sorted(set(b[0] for b in correlations.keys()))
    bob_bases = sorted(set(b[1] for b in correlations.keys()))
    corr_matrix = np.zeros((len(alice_bases), len(bob_bases)))
    
    for i, a_base in enumerate(alice_bases):
        for j, b_base in enumerate(bob_bases):
            if (a_base, b_base) in correlations:
                corr_matrix[i, j] = correlations[(a_base, b_base)]
    
    colors = [(0.8, 0.2, 0.2), (1, 1, 1), (0.2, 0.2, 0.8)]
    cmap = LinearSegmentedColormap.from_list('rwb', colors, N=100)
    
    im = ax1.imshow(corr_matrix, cmap=cmap, vmin=-1, vmax=1)
    ax1.set_title('Correlation Values E(a,b)')
    ax1.set_xticks(np.arange(len(bob_bases)))
    ax1.set_yticks(np.arange(len(alice_bases)))
    ax1.set_xticklabels([f"{b}°" for b in bob_bases])
    ax1.set_yticklabels([f"{a}°" for a in alice_bases])
    ax1.set_xlabel("Bob's Angle")
    ax1.set_ylabel("Alice's Angle")
    
    for i in range(len(alice_bases)):
        for j in range(len(bob_bases)):
            ax1.text(j, i, f"{corr_matrix[i, j]:.2f}", ha="center", va="center",
                    color="black" if abs(corr_matrix[i, j]) < 0.5 else "white")
    
    fig.colorbar(im, ax=ax1)
    
    # CHSH bar chart
    ax2 = fig.add_subplot(122)
    ax2.bar([0], [chsh_value], width=0.4, color='purple', alpha=0.7)
    ax2.axhline(y=2.0, color='r', linestyle='-', label='Classical Limit (2.0)')
    ax2.axhline(y=2*np.sqrt(2), color='b', linestyle='--', label='Quantum Limit (2√2)')
    ax2.set_ylim(0, 3.0)
    ax2.set_xticks([0])
    ax2.set_xticklabels(['CHSH Value'])
    ax2.text(0, chsh_value + 0.1, f"{chsh_value:.3f}", ha='center')
    ax2.legend(loc='upper left')
    ax2.set_title('CHSH Value vs Limits')
    
    verdict = " Entanglement!" if chsh_value > 2 else " No quantum"
    ax2.text(0, 1.0, verdict, ha='center', fontsize=14, fontweight='bold',
             color='green' if chsh_value > 2 else 'red')
    
    plt.suptitle(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

---
## Lancer le test de Bell complet !

Mettons tout cela ensemble et vérifions l’intrication quantique.


In [ ]:
# RUN THE COMPLETE BELL TEST

# Create 1000 Bell pairs (Alice and Bob share each pair)
NUM_PAIRS = 400
print(f"\nCreating {NUM_PAIRS} Bell pairs...")
bell_pairs = [create_bell_pair_singlet_state() for _ in range(NUM_PAIRS)]

# Run measurements on all pairs
print("Running measurements...")
results, alice_bases_used, bob_bases_used = run_bell_test_measurements(bell_pairs)

# Organize by basis pairs for correlation calculation
print("\nOrganizing results...")
organized = organize_measurements_by_basis(results, alice_bases_used, bob_bases_used)

# Calculate correlations
print("\nCalculating correlations...")
correlations = calculate_correlations(organized)

# Calculate CHSH
chsh = calculate_chsh_value(correlations)
print(f"\n" + "="*60)
print(f"CHSH VALUE: {chsh:.4f}")
print("="*60)

if check_bell_inequality(chsh):
    print("\n QUANTUM ENTANGLEMENT DETECTED!")
    print("   The Bell inequality is VIOLATED.")
    print("   This cannot be explained by classical physics!")
else:
    print("\n No quantum correlations detected.")
    print("   Check your implementation.")

In [ ]:
# Visualize
visualize_bell_test_results(correlations, chsh, "Bell Test: Singlet State |Ψ-⟩")

---
## Défi : Détecter l'espionnage !

Lorsque Eve intercepte et mesure les qubits, elle détruit l'intrication.  
Elle doit donc **envoyer un qubit de remplacement** à Bob (sinon il remarquerait les photons manquants).  
Dans ce modèle simplifié, elle **mesure**, puis **recrée un nouvel état produit** correspondant à son résultat de mesure.

Voyons comment cela affecte la valeur CHSH.


In [ ]:
def create_eavesdropped_state(bell_qc):
    """
    Simulate Eve measuring the Bell pair (destroys entanglement).
    """
    # Eve measures in Z basis
    eve_qc = bell_qc.copy()
    eve_qc.measure_all()
    eve_result = run_circuit(eve_qc, shots=1)
    eve_bits = list(eve_result.keys())[0]
    
    # Eve recreates a product state based on her measurement
    qc = QuantumCircuit(2)
    if eve_bits[0] == '1':  # qubit 1
        qc.x(1)
    if eve_bits[1] == '1':  # qubit 0
        qc.x(0)
    
    return qc

## Test avec Eve interceptant 70 % des paires

In [ ]:
# Test with Eve intercepting 70% of pairs
EVE_PERCENTAGE = 0.7

print(f"\n EAVESDROPPING SIMULATION")
print(f"Eve intercepts {EVE_PERCENTAGE*100}% of Bell pairs\n")

# Create Bell pairs
print(f"Create Bell {NUM_PAIRS} pairs ...")
bell_pairs = [create_bell_pair_singlet_state() for _ in range(NUM_PAIRS)]

# Eve compromises some pairs
compromised = int(NUM_PAIRS * EVE_PERCENTAGE)
print(f"Number of compromised {compromised} / {NUM_PAIRS} ...")
eve_pairs = [create_eavesdropped_state(qc) for qc in bell_pairs[:compromised]] + bell_pairs[compromised:]

# Run Bell test on compromised pairs
print("Run circuit and get measurements ...")
results, alice_bases_used, bob_bases_used = run_bell_test_measurements(eve_pairs)

# Organize, calculate correlations and CHSH
organized = organize_measurements_by_basis(results, alice_bases_used, bob_bases_used)
correlations = calculate_correlations(organized)
chsh = calculate_chsh_value(correlations)

print(f"\nCHSH VALUE with Eve: {chsh:.4f}")

if check_bell_inequality(chsh):
    print("\n QUANTUM CORRELATIONS DETECTED!")
    print("   The system is entangled. The channel appears secure.")
    if EVE_PERCENTAGE > 0:
        print(f"   (Note: Eve was present at {EVE_PERCENTAGE*100}%, but didn't intercept enough to break the inequality.)")
else:
    print("\n EAVESDROPPING DETECTED!")
    print("   The Bell inequality is satisfied (S ≤ 2.0).")
    print("   Alice and Bob should abort: the connection is NOT secure.")


 EAVESDROPPING SIMULATION
Eve intercepts 70.0% of Bell pairs



NameError: name 'NUM_PAIRS' is not defined

In [ ]:

visualize_bell_test_results(correlations, chsh, f"Bell Test: {EVE_PERCENTAGE*100}% Eavesdropped")


<br><br><br>
# ==========================================
# PARTIE 2 : LE PROTOCOLE E91
# ==========================================


# Protocole de distribution de clé quantique E91 – Atelier pratique

**Objectifs d'apprentissage :**
- Implémenter le protocole E91 complet
- Utiliser l'inégalité de Bell CHSH pour détecter l'espionnage
- Générer une clé partagée sécurisée
- Chiffrer et déchiffrer des messages

---

## Rappel rapide (des diapositives)

**Étapes du protocole E91 :**
1. Générer des paires de Bell intriquées
2. Alice et Bob choisissent au hasard leurs bases de mesure
3. Mesurer toutes les paires
4. Trier les résultats : mêmes bases → clé, bases CHSH → test de sécurité
5. Lancer le test CHSH : si $|S| > 2$ → sécurisé !
6. Extraire la clé à partir des mesures dans les bases concordantes

---


---
## Points clés du protocole E91 (Rappel)

- **Bases :** Alice $\{\color{red}{0^\circ}$, $45^\circ$, $\color{red}{90^\circ}\}$, Bob $\{\color{red}{45^\circ}$, $90^\circ$, $\color{red}{135^\circ}\}$
- **Paires pour la clé :** $(45^\circ, 45^\circ)$ et $(90^\circ, 90^\circ)$
- **Paires pour le test CHSH :** $(0^\circ, 90^\circ)$, $(0^\circ, 135^\circ)$, $(45^\circ, 90^\circ)$, $(45^\circ, 135^\circ)$
- **Rejeter :** toutes les autres combinaisons de bases
- **Sécurité :** calculer $S$ à partir des paires CHSH ; sécurisé si $|S| > 2$
- **État singulet :** les résultats sont anti-corrélés, donc Bob inverse les bits pour la clé
---


## Configuration

**Important :** assurez-vous d'avoir d'abord complété et exécuté **toutes les cellules** du premier notebook !

**Rappel :** la plupart des fonctions auxiliaires vous sont déjà familières depuis le CHSH ; l'**étape de tri** est le cœur du protocole E91, nous allons donc nous concentrer sur celle-ci !


In [ ]:
import random
import numpy as np
from qiskit import QuantumCircuit
import sys
sys.path.append('utils')
import encryption_algorithms as enc

# Import directly from Notebook 1!
# This imports YOUR functions that you implemented
import import_ipynb

# Suppress notebook output during import
import io
_stdout = sys.stdout
sys.stdout = io.StringIO()

# Import your completed CHSH notebook as a module
from importlib import import_module
notebook1 = __import__("01_CHSH_Bell_Inequality")

# Restore output
sys.stdout = _stdout

# Now all your functions are available!
run_circuit = notebook1.run_circuit
create_bell_pair_singlet_state = notebook1.create_bell_pair_singlet_state
apply_basis_transformation = notebook1.apply_basis_transformation
measure_bell_pair = notebook1.measure_bell_pair
create_eavesdropped_state = notebook1.create_eavesdropped_state
organize_measurements_by_basis = notebook1.organize_measurements_by_basis
calculate_correlations = notebook1.calculate_correlations
calculate_chsh_value = notebook1.calculate_chsh_value

print(" Setup complete!")
print("\n Imported YOUR functions from Notebook 1:")
print("   • run_circuit")
print("   • create_bell_pair_singlet_state") 
print("   • apply_basis_transformation")
print("   • measure_bell_pair")
print("   • create_eavesdropped_state")
print("   • organize_measurements_by_basis")
print("   • calculate_correlations")
print("   • calculate_chsh_value")

# Set seed for reproducibility  
GLOBAL_SEED = 91
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

# E91 Protocol Constants
ALICE_BASES = ['0', '45', '90']      # Alice's available bases
BOB_BASES = ['45', '90', '135']      # Bob's available bases
ALICE_CHSH_BASES = ['0', '90']
BOB_CHSH_BASES = ['45', '135']
CHSH_BASIS_PAIRS = [(a, b) for a in ALICE_CHSH_BASES for b in BOB_CHSH_BASES]

BELL_INEQUALITY_THRESHOLD = 2.0
EVE_PERCENTAGE_COMPROMISED = 0.7 # Eve intercepts 70% of the pairs

print(f"\nAlice's bases: {ALICE_BASES}")
print(f"Bob's bases: {BOB_BASES}")
print(f"Key generation pairs: (45,45) and (90,90)")

AttributeError: 'NoneType' object has no attribute 'draw'

## 1) Générer des bases aléatoires

Alice et Bob choisissent chacun, pour chaque qubit, une base de mesure de manière aléatoire.  
Pourquoi : des choix indépendants et aléatoires empêchent Eve de prédire les paramètres de mesure.


In [2]:
# Test
test_bases = generate_random_bases(10, ['0', '45', '90'])
print(f"Generated bases: {test_bases}")
print(f"Expected: 10 random values from ['0', '45', '90']")

NameError: name 'random' is not defined

## 2) Créer des paires de Bell

Pourquoi : le protocole E91 commence par générer de nombreuses paires intriquées identiques (même principe que pour le CHSH).


## 3) Mesurer toutes les paires

## 4) Trier les résultats (Génération de clé vs Test de Bell)

**Règles de tri :**
- Mêmes bases (45,45) ou (90,90) → **Génération de clé**
- Bases CHSH → **Test de Bell**
- Autres combinaisons → **Rejet**

**C'est le cœur du protocole E91** — cela détermine quelles données deviennent la clé et lesquelles servent au test de sécurité.

Exercice : implémentez `extract_e91_key_and_bell_test_data()`


### Intuition du tri (pourquoi cela fonctionne)

- **Mêmes bases** → forte (anti-)corrélation → sûr pour les bits de la clé
- **Bases différentes** → plus aléatoires → utile pour le test de Bell ou à rejeter

Cela reflète ce qui se passe avec les polariseurs dans l’expérience en laboratoire.


In [ ]:
def extract_e91_key_and_bell_test_data(results: list, alice_bases: list, bob_bases: list) -> dict:
    """
    Sift measurement results into key generation and Bell test data.
    
    Returns:
        dict with:
            'key_results': results for key generation
            'chsh_results': results for Bell test
            'chsh_alice_bases': Alice's bases for Bell test
            'chsh_bob_bases': Bob's bases for Bell test
    """
    key_results = [] # keep output results for same bases
    chsh_results = [] # keep output results for bases combinations that belong to CHSH test
    chsh_alice_bases = [] # keep alice bases that are in (combinations that belong to CHSH test)
    chsh_bob_bases = [] # same as alice
    # note : at index i, we have chsh output related to alice and bob base
    
    for i, (result, a_base, b_base) in enumerate(zip(results, alice_bases, bob_bases)):
        
        # TODO: Check if same basis (a_base == b_base)
        # If yes, add to key_results
        
        # TODO: Check if (a_base, b_base) is in CHSH_BASIS_PAIRS
        # If yes, add to chsh_results, chsh_alice_bases, chsh_bob_bases
        
        # Otherwise: discard
        pass
    
    return {
        'key_results': key_results,
        'chsh_results': chsh_results,
        'chsh_alice_bases': chsh_alice_bases,
        'chsh_bob_bases': chsh_bob_bases,
    }

## 5) Vérifier l'espionnage

Utilisez les données du test de Bell pour vérifier la sécurité.  
Si $|S| > 2$, l'intrication est intacte et le canal est sécurisé.

*(Cette cellule réutilise la logique CHSH du notebook 1. Fournie pour gagner du temps.)*


In [ ]:
def check_for_eavesdropping(chsh_results: list, chsh_alice_bases: list, chsh_bob_bases: list) -> dict:
    """
    Run CHSH test to check for eavesdropping.
    """
    # 1. Organize the raw measurement results by basis pair
    bell_results = organize_measurements_by_basis(chsh_results, chsh_alice_bases, chsh_bob_bases)
    
    # 2. Calculate the correlation for each basis pair
    correlations = calculate_correlations(bell_results)
    
    # 3. Compute the CHSH 'S' value
    chsh_value = calculate_chsh_value(correlations)
    
    # 4. Check against the threshold (2.0)
    is_secure = chsh_value > BELL_INEQUALITY_THRESHOLD
    
    return {
        'chsh_value': chsh_value,
        'is_secure': is_secure
    }


## 6) Protocole E91 complet

Mettons tout cela ensemble !  
Cette orchestration montre l’ensemble du processus, de l’intrication jusqu’à l’obtention d’une clé partagée.

Exercice : complétez `run_e91_protocol()`


In [ ]:
EVE_PERCENTAGE_COMPROMISED = 0.7

def run_e91_protocol(num_pairs: int = 2000, eavesdropping: bool = False) -> str:
    """
    Run the complete E91 QKD protocol.
    
    Args:
        num_pairs: Number of Bell pairs to generate
        eavesdropping: If True, simulate Eve intercepting pairs
    
    Returns:
        Shared secret key (string of 0s and 1s) or None if insecure
    """
    print("="*60)
    print("E91 QUANTUM KEY DISTRIBUTION PROTOCOL")
    print("="*60)
    
    # Step 1: Generate Bell pairs
    print(f"\n1. Generating {num_pairs} Bell pairs...")
    bell_pairs = create_list_bell_pairs(num_pairs)
    
    # Step 2: Simulate eavesdropping (optional)
    if eavesdropping:
        compromised = int(num_pairs * EVE_PERCENTAGE_COMPROMISED)
        print(f"\n EVE intercepts {compromised} pairs ({EVE_PERCENTAGE_COMPROMISED*100}%)")
        bell_pairs = [create_eavesdropped_state(qc) for qc in bell_pairs[:compromised]] + bell_pairs[compromised:]
    
    # Step 3: Generate random bases
    print("\n2. Alice and Bob choose random bases...")
    alice_bases = generate_random_bases(num_pairs, ALICE_BASES)
    bob_bases = generate_random_bases(num_pairs, BOB_BASES)
    
    # Step 4: Measure all pairs
    print("\n3. Measuring all pairs...")
    results = measure_all_pairs(bell_pairs, alice_bases, bob_bases)
    
    # Step 5: Sift results
    print("\n4. Sifting results...")
    data = extract_e91_key_and_bell_test_data(results, alice_bases, bob_bases)
    
    print(f"   Key generation pairs: {len(data['key_results'])}")
    print(f"   Bell test pairs: {len(data['chsh_results'])}")
    
    # Step 6: Security check
    print("\n5. Running CHSH security test...")
    security = check_for_eavesdropping(
        data['chsh_results'],
        data['chsh_alice_bases'],
        data['chsh_bob_bases']
    )
    
    print(f"\n   CHSH Value: {security['chsh_value']:.4f}")
    print(f"   Classical limit: 2.0")
    
    if not security['is_secure']:
        print("\n SECURITY CHECK FAILED!")
        print("   Possible eavesdropping detected.")
        print("   Key generation ABORTED.")
        return None
    
    print("\n SECURITY CHECK PASSED!")
    
    # Step 7: Extract key
    print("\n6. Extracting shared key...")
    
    # Alice's bits (first bit of each result)
    alice_key = ''.join([r[0] for r in data['key_results']])
    
    # Bob's bits (second bit of each result)
    # For singlet state: anti-correlated, so Bob flips his bits
    bob_key = ''.join(['1' if r[1] == '0' else '0' for r in data['key_results']])
    
    # Verify keys match
    if alice_key != bob_key:
        mismatches = sum(a != b for a, b in zip(alice_key, bob_key))
        print(f"\n Warning: {mismatches} bit mismatches (noise or Eve)")
    else:
        print("\n Keys match perfectly!")
    
    print(f"\n   Key length: {len(alice_key)} bits")
    print(f"   Key (first 50 bits: {alice_key[:50]}...")
    
    return alice_key


## Lancer le protocole E91 (sans espionnage)

In [ ]:
# Run without Eve
key = run_e91_protocol(num_pairs=2000, eavesdropping=False)

## Lancer le protocole E91 (avec espionnage)

In [ ]:
# Reset seed
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)
MANUAL_SIMULATOR_SEED_COUNTER = GLOBAL_SEED

# Run with Eve
key_with_eve = run_e91_protocol(num_pairs=2000, eavesdropping=True)

## Chiffrer et déchiffrer des messages

In [ ]:
# use the generated key to encrypt and decrypt a message using XOR
# I use the key generated without Eve for encryption, as the one with Eve is likely compromised
message = '"What I cannot create, I do not understand." - Richard Feynman'
print(f"\nOriginal: {message}")

encrypted = enc.encrypt_xor_repeating_key(message, key)
print(f"\nEncrypted (hex): {encrypted[:50]}...")

decrypted = enc.decrypt_xor_repeating_key(encrypted, key)
print(f"\nDecrypted: {decrypted}")

---
## Défi : Déchiffrer les messages secrets !

Le fichier `encrypted_messages.txt` contient des messages chiffrés à l’aide de la clé E91.  
**Votre mission : les déchiffrer !**


In [ ]:
def decrypt_messages_from_file(key: str, filename: str):
    """Decrypt all messages from a file."""
    print(f"\nDecrypting messages from {filename}...\n")
    
    try:
        with open(filename, 'r') as f:
            for line in f:
                if ': ' in line:
                    msg_id, encrypted = line.strip().split(': ', 1)
                    decrypted = enc.decrypt_xor_repeating_key(encrypted, key)
                    print(f"{msg_id}: {decrypted}")
    except FileNotFoundError:
        print(f" File not found: {filename}")


# Decrypt the messages
if key:
    decrypt_messages_from_file(key, 'encrypted_messages.txt')

---
## Résumé

**Ce que nous avons implémenté :**
1. Protocole complet de distribution de clé quantique E91
2. Sélection aléatoire des bases pour Alice et Bob
3. Tri des résultats (clé vs test de Bell)
4. Vérification de la sécurité CHSH
5. Extraction de la clé avec gestion de l’anti-corrélation
6. Chiffrement et déchiffrement de messages

**Aperçu clé :** le test de Bell CHSH détecte l’espionnage !
- Pas d’Eve → $|S| \approx 2,83$ → Sécurisé 
- Avec Eve → $|S|$ chute → Détecté ! 

---

**Ensuite :** complétez le devoir à la maison en utilisant un **état de Bell différent** !



<br><br><br>
# ==========================================
# PARTIE 3 : DÉFI DE CODAGE (DEVOIR)
# ==========================================


# Défi de programmation - E91 avec |Φ-⟩

Lors de l'atelier, nous avons implémenté le protocole E91 en utilisant l'**état singulet** |Ψ-⟩ = (|01⟩ - |10⟩)/√2.

Dans ce devoir, vous allez adapter l'E91 pour utiliser l'**état de Bell |Φ-⟩** = (|00⟩ - |11⟩)/√2.

**Différence clé :**
- |Ψ-⟩ donne des résultats **anti-corrélés** (Alice=0 → Bob=1)  
- |Φ-⟩ donne des résultats **corrélés** (Alice=0 → Bob=0)

## Ce que vous allez apprendre :

1. **Tout état de Bell fonctionne** pour l'E91, mais il faut utiliser la bonne formule !  
2. Les états **corrélés vs anti-corrélés** influencent l'extraction de la clé  
3. **La formule CHSH change** selon l'état de Bell utilisé  
4. **Approche expérimentale** : essayez toutes les possibilités et trouvez celle qui donne $2\sqrt{2}$

**À FAIRE :** Adaptez le protocole E91 pour utiliser l'état de Bell |Φ-⟩ et déchiffrez les messages secrets.  
- Créez le circuit |Φ-⟩ (astuce : ajoutez UNE porte à |Φ+⟩)  
- Trouvez la formule CHSH correcte (essayez les 4 positions du signe moins !)  
- Complétez le protocole E91 et déchiffrez les messages


### Configuration


## Tâche 1 : Créer l'état de Bell |Φ-⟩ (15 points)

Complétez la fonction pour créer |Φ-⟩ = (|00⟩ - |11⟩)/√2

**Astuce :** Commencez par |Φ+⟩ et ajoutez UNE porte pour créer le signe moins !

```
|Φ+⟩: H → CX       gives (|00⟩ + |11⟩)/√2
|Φ-⟩: H → ? → CX   gives (|00⟩ - |11⟩)/√2
```

**Rappel :** La porte Z ajoute une phase π : |1⟩ → -|1⟩


In [ ]:
def create_bell_pair_phi_minus() -> QuantumCircuit:
    """
    TODO: Create the Bell state |Φ-⟩ = (|00⟩ - |11⟩)/√2
    
    Hint: Start with |Φ+⟩ (H + CX) and add ONE gate to create the minus sign!
    """
    qc = QuantumCircuit(2)
    
    # TODO: Add your gates here
    # qc.h(0)     # Hadamard on qubit 0
    # qc.?        # What gate creates the minus sign?
    # qc.cx(0,1)  # CNOT
    
    return qc

In [ ]:
# Test your circuit
phi_minus = create_bell_pair_phi_minus()
print("Your circuit for |Φ-⟩:")
print(phi_minus.draw())

### Vérification de l'état |Φ-⟩

Assurez-vous que vous avez bien créé le bon état de Bell.  
Astuce : $1/\sqrt{2}$ est identique à $\sqrt{2}/2$ 😄


In [ ]:
from qiskit.quantum_info import Statevector
Statevector(phi_minus).draw('latex')

## Fonctions auxiliaires (fournies)

Ce sont les mêmes fonctions que celles de l'atelier. Exécutez cette cellule !


## Vérification : Testez votre circuit |Φ-⟩

Exécutez ceci pour vérifier si votre circuit est correct. Vous devriez voir environ 50 % de |00⟩ et 50 % de |11⟩.


In [ ]:
# Test the |Φ-⟩ Bell state
phi_minus = create_bell_pair_phi_minus()

# Verify with measurements
test_qc = phi_minus.copy()
test_qc.measure_all()
counts = run_circuit(test_qc, shots=1000)

print(f"\nMeasurement results: {counts}")
print("\n Expected: ~50% |00⟩ and ~50% |11⟩ (never |01⟩ or |10⟩)")
print("   This confirms |Φ-⟩ is CORRELATED : ")
print("   --> Alice and Bob get SAME results: Alice=0 → Bob=0, Alice=1 → Bob=1")

print("\n Note: The minus sign (phase) doesn't affect Z-basis measurements,")
print("   but it DOES affect measurements in rotated bases (used for CHSH)!")

## Tâche 2 : Trouver la bonne formule CHSH

La formule CHSH contient UN signe moins. Pour |Φ-⟩, il se déplace vers une position différente !

**Formule de l'atelier (pour |Ψ-⟩) :**
$$S = E(a,b) - E(a,b') + E(a',b) + E(a',b')$$

**Votre tâche :** Essayez les 4 positions pour le signe moins et trouvez celle qui donne |S| ≈ 2√2 ≈ 2,83


**D’abord, nous générons des données de test pour |Φ⁻⟩ :**

In [ ]:
# Generate test data for |Φ-⟩

print("Generating 500 measurements for CHSH test...\n")
test_num_pairs = 500

# Generate random bases
test_alice_chsh_bases = [random.choice(ALICE_CHSH_BASES) for _ in range(test_num_pairs)]
test_bob_chsh_bases = [random.choice(BOB_CHSH_BASES) for _ in range(test_num_pairs)]

# Generate Bell pairs and Measure them
results = []
for a, b in zip(test_alice_chsh_bases, test_bob_chsh_bases):
    qc = create_bell_pair_phi_minus()
    results.append(measure_bell_pair(qc, a, b))


# step 4: Organize measurements by basis to compute correkations:
measurements = organize_measurements_by_basis(results, test_alice_chsh_bases, test_bob_chsh_bases)
correlations = calculate_correlations(measurements)

# Print correlations
print("Individual correlations E(a,b):")
for pair, E in sorted(correlations.items()):
    print(f"  E{pair} = {E:.4f}")

# Step 5: TODO
# Based on the correlations, compute and find the correct CHSH formulat (in next cell)

In [ ]:
# ═══════════════════════════════════════════════════════════
# TODO: Try all 4 minus sign positions!
# ═══════════════════════════════════════════════════════════

a1, a2 = '0', '90'   # Alice's CHSH bases
b1, b2 = '45', '135' # Bob's CHSH bases

E_ab = correlations[(a1, b1)]              # E(0°, 45°)
E_ab_prime = correlations[(a1, b2)]        # E(0°, 135°)
E_a_prime_b = correlations[(a2, b1)]       # E(90°, 45°)
E_a_prime_b_prime = correlations[(a2, b2)] # E(90°, 135°)

print("="*60)
print("Testing all 4 CHSH formula variations:")
print("="*60)

# TODO: Calculate S for each position of the minus sign

# Position 1: minus at first term
S_minus_p1 = 0  # TODO: -E_ab + E_ab_prime + E_a_prime_b + E_a_prime_b_prime
print(f"\n1. S = -E(a,b) + E(a,b') + E(a',b) + E(a',b')")
print(f"   |S| = {abs(S_minus_p1):.4f}")

# Position 2: minus at second term (original workshop formula)
S_minus_p2 = 0  # TODO: E_ab - E_ab_prime + E_a_prime_b + E_a_prime_b_prime
print(f"\n2. S = +E(a,b) - E(a,b') + E(a',b) + E(a',b')  [workshop formula]")
print(f"   |S| = {abs(S_minus_p2):.4f}")

# Position 3: minus at third term
S_minus_p3 = 0  # TODO: E_ab + E_ab_prime - E_a_prime_b + E_a_prime_b_prime
print(f"\n3. S = +E(a,b) + E(a,b') - E(a',b) + E(a',b')")
print(f"   |S| = {abs(S_minus_p3):.4f}")

# Position 4: minus at fourth term
S_minus_p4 = 0  # TODO: E_ab + E_ab_prime + E_a_prime_b - E_a_prime_b_prime
print(f"\n4. S = +E(a,b) + E(a,b') + E(a',b) - E(a',b')")
print(f"   |S| = {abs(S_minus_p4):.4f}")

print("\n" + "="*60)
print(f"Target: 2√2 ≈ {2*np.sqrt(2):.4f}")
print("The formula with |S| ≈ 2.83 is the CORRECT one!")
print("="*60)

### Votre réponse : Quelle formule CHSH est correcte ?

D'après vos résultats ci-dessus, indiquez la formule CHSH correcte pour |Φ-⟩ :


In [ ]:
def calculate_chsh_value_phi_minus(correlations, alice_bases=ALICE_CHSH_BASES, bob_bases=BOB_CHSH_BASES):
    """
    TODO: Implement the correct CHSH formula for |Φ-⟩
    
    Based on your experiment above, which position gives |S| ≈ 2√2?
    """
    a1, a2 = alice_bases  # '0', '90'
    b1, b2 = bob_bases    # '45', '135'
    
    # TODO: Write the correct formula here
    # S = ???
    S = 0  # Replace with the correct formula!
    
    
    return abs(S)


# Test your formula
chsh = calculate_chsh_value_phi_minus(correlations)
print(f"Your CHSH value: {chsh:.4f}")
print(f"Expected: ≈ {2*np.sqrt(2):.4f}")

## Fonctions auxiliaires du protocole E91

Ces fonctions orchestrent l’ensemble du protocole de distribution de clé E91.


In [ ]:
# ============================================================
# PART 2: E91 PROTOCOL - Helper Functions
# ============================================================

def generate_random_bases(length, options):
    """Generate a list of random measurement bases."""
    return [random.choice(options) for _ in range(length)]


def create_list_bell_pairs(num_pairs):
    """Create a list of |Φ-⟩ Bell pairs."""
    return [create_bell_pair_phi_minus() for _ in range(num_pairs)]


def measure_all_pairs(bell_pairs, alice_bases, bob_bases):
    """Measure all Bell pairs with the specified bases."""
    results = []
    for qc, a_base, b_base in zip(bell_pairs, alice_bases, bob_bases):
        result = measure_bell_pair(qc, a_base, b_base)
        results.append(result)
    return results


def extract_e91_key_and_bell_test_data(results, alice_bases, bob_bases):
    """
    Sift measurement results into key generation and Bell test data.
    
    Rules:
    - Same basis (45,45) or (90,90) → Key generation
    - CHSH basis pairs → Bell test
    - Other combinations → Discard
    """
    key_results = []
    chsh_results = []
    chsh_alice_bases = []
    chsh_bob_bases = []
    
    for result, a_base, b_base in zip(results, alice_bases, bob_bases):
        if a_base == b_base:  # Same basis → Key
            key_results.append(result)
        elif (a_base, b_base) in CHSH_BASIS_PAIRS:  # CHSH bases → Security test
            chsh_results.append(result)
            chsh_alice_bases.append(a_base)
            chsh_bob_bases.append(b_base)
        # else: discard
    
    return {
        'key_results': key_results,
        'chsh_results': chsh_results,
        'chsh_alice_bases': chsh_alice_bases,
        'chsh_bob_bases': chsh_bob_bases,
    }


def check_for_eavesdropping(chsh_results, chsh_alice_bases, chsh_bob_bases):
    """Run CHSH security test on Bell test data."""
    bell_results = organize_measurements_by_basis(chsh_results, chsh_alice_bases, chsh_bob_bases)
    correlations = calculate_correlations(bell_results)
    chsh_value = calculate_chsh_value_phi_minus(correlations)
    
    return {
        'chsh_value': chsh_value,
        'is_secure': chsh_value > BELL_INEQUALITY_THRESHOLD
    }


print(" Part 2: E91 Protocol functions loaded!")


## Tâche 3 : Compléter le protocole E91 et déchiffrer les messages 
Maintenant, mettez tout cela ensemble ! Complétez la fonction `run_e91_protocol`.

**Rappel :**
- Utilisez votre état de Bell |Φ-⟩
- Utilisez votre formule CHSH correcte
- |Φ-⟩ est **corrélé** → Bob utilise les bits directement (pas d'inversion !)

In [ ]:
# ============================================================
# COMPLETE E91 PROTOCOL with |Φ-⟩
# ============================================================

# Reset seed for reproducibility
MANUAL_SIMULATOR_SEED_COUNTER = 42
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

def run_e91_protocol(num_pairs=2000):
    """
    Run E91 protocol with |Φ-⟩ Bell state.
    
    TODO: Complete this function using:
    - Your create_bell_pair_phi_minus() function
    - Your calculate_chsh_value_phi_minus() function
    - Correct key extraction for CORRELATED state
    """
    print("="*60)
    print("E91 PROTOCOL with |Φ-⟩ Bell State")
    print("="*60)
    
    # Step 1: Generate Bell pairs
    print(f"\n1. Generating {num_pairs} |Φ-⟩ Bell pairs...")
    bell_pairs = create_list_bell_pairs(num_pairs)
    
    # Step 2: Generate random bases
    print("\n2. Alice and Bob choose random bases...")
    alice_bases = generate_random_bases(num_pairs, ALICE_BASES)
    bob_bases = generate_random_bases(num_pairs, BOB_BASES)
    
    # Step 3: Measure all pairs
    print("\n3. Measuring all pairs...")
    results = measure_all_pairs(bell_pairs, alice_bases, bob_bases)
    
    # Step 4: Sift results
    print("\n4. Sifting results...")
    data = extract_e91_key_and_bell_test_data(results, alice_bases, bob_bases)
    print(f"   Key generation pairs: {len(data['key_results'])}")
    print(f"   CHSH test pairs: {len(data['chsh_results'])}")
    
    # Step 5: Security check
    print("\n5. Running CHSH security test...")
    security = check_for_eavesdropping(
        data['chsh_results'],
        data['chsh_alice_bases'],
        data['chsh_bob_bases']
    )
    print(f"\n   CHSH Value: {security['chsh_value']:.4f}")
    print(f"   Classical limit: 2.0")
    print(f"   Quantum limit: 2√2 ≈ 2.83")
    
    if not security['is_secure']:
        print("\n SECURITY CHECK FAILED!")
        print("   Possible eavesdropping detected.")
        return None
    
    print("\n SECURITY CHECK PASSED!")
    
    # Step 6: Extract key
    print("\n6. Extracting shared key...")
    
    # TODO: Key extraction for |Φ-⟩
    # Remember: |Φ-⟩ is CORRELATED → Bob uses bits DIRECTLY (no flip!)
    # Qiskit format: result = "BA" where B=qubit1, A=qubit0
    
    alice_key = ''.join([r[1] for r in data['key_results']])  # Alice = qubit 0
    bob_key = None    ## r[0] = Bob (qubit 1) - TODO: flip or not?

    
    if alice_key == bob_key:
        print("\n Keys match perfectly!")
    else:
        mismatches = sum(a != b for a, b in zip(alice_key, bob_key))
        print(f"\n {mismatches} mismatches in {len(alice_key)} bits")
    
    print(f"   Key length: {len(alice_key)} bits")
    print(f"   Key (first 50): {alice_key[:50]}...")
    
    return alice_key


print(" E91 Protocol function loaded!")

### Exécuter le protocole E91


In [ ]:
key = run_e91_protocol(num_pairs=2000)

### Déchiffrement des messages secrets

Si votre protocole a été exécuté correctement, vous devriez pouvoir déchiffrer les messages !


In [ ]:
def decrypt_and_print_messages(key: str, filename: str = "encrypted_messages.txt"):
    """
    Read encrypted messages from a file, decrypt them using XOR with the given key, and print.

    Args:
        key (str): The key to use for XOR decryption.
        filename (str): The file to read encrypted messages from.
    """
    print("\nDecrypting all messages from", filename)
    with open(filename, "r") as f:
        for line in f:
            if ": " in line:
                msg_id, encrypted = line.split(": ", 1)
                decrypted = enc.decrypt_xor_repeating_key(encrypted.strip(), key)
                print(f"{msg_id}: {decrypted}")


In [ ]:
# For your challenge: must generate the correct key using the E91 protocol to decrypt the messages. Without the key, decryption is not feasible.
# change the path to the encrypted file as needed
path_to_encrypted_file = R"assignment_encrypted_messages.txt"
decrypt_and_print_messages(key, filename=path_to_encrypted_file)


## Résumé : Ce que vous avez appris

Complétez ce tableau en fonction de votre travail :

| Aspect             | Atelier ($\vert \Psi^- \rangle$)           | Ce Devoir ($\vert \Phi^- \rangle$) |
|-------------------|-------------------------------------------|------------------------------------|
| Circuit            | H → CX → X → Z                             | H → ??? → CX                       |
| Corrélation        | Anti-corrélée                              | ???                                |
| Extraction de clé  | Bob inverse les bits                        | ???                                |
| Formule CHSH       | $S = +E(a,b) - E(a,b') + E(a',b) + E(a',b')$ | $S = ???$                          |

**Aperçu clé :** N'importe quel état de Bell peut être utilisé pour l'E91, mais vous devez :  
1. Savoir s'il est **corrélé** ou **anti-corrélé** (pour l'extraction de la clé)  
2. Trouver la **bonne formule CHSH** (celle qui donne $|S| \approx 2\sqrt{2}$)
